In [32]:
from qdrant_client import AsyncQdrantClient, models
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import AsyncQdrantClient
import torch.nn.functional as F
from tools import is_text
import torch
from transformers import AutoTokenizer, AutoModel
import sys
import os
from itertools import count
from llama_index.core import VectorStoreIndex
import json
from tqdm import tqdm
from smart_chunker.chunker import SmartChunker
import pickle
from typing import List
import httpx

In [ ]:
REST_API_PORT=6333
EMBEDDINGS_DIM=768
model_name = "gemma-3-27b-it"

In [34]:
docs_config_dir = "doc_base/RuBQ_2.0/"
test_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_test.json')
dev_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_dev.json')
paragraphs_path = os.path.join(docs_config_dir, 'RuBQ_2.0_paragraphs.json')

In [35]:
with open(test_queries_path, 'r', encoding='utf-8') as f:
    test_queries = json.load(f)

with open(dev_queries_path, 'r', encoding='utf-8') as f:
    dev_queries = json.load(f)

with open(paragraphs_path, 'r', encoding='utf-8') as f:
    paragraphs = json.load(f)

In [36]:
dev_queries

[{'uid': 4,
  'question_text': 'Какой стране принадлежит знаменитый остров Пасхи?',
  'query': 'SELECT ?answer \nWHERE {\n  wd:Q14452 wdt:P17 ?answer\n}',
  'answer_text': 'Чили',
  'question_uris': ['http://www.wikidata.org/entity/Q14452'],
  'question_props': ['wdt:P17'],
  'answers': [{'type': 'uri',
    'label': 'Чили',
    'value': 'http://www.wikidata.org/entity/Q298',
    'wd_names': {'ru': ['Республика Чили', 'Чили'],
     'en': ['Chile',
      'República de Chile',
      'Republic of Chile',
      'cl',
      'Republica de Chile',
      'CHI',
      '🇨🇱']},
    'wp_names': ['Чилийская']}],
  'paragraphs_uids': {'with_answer': [10785, 10782, 10783],
   'all_related': [10782,
    10783,
    10784,
    10785,
    10786,
    53027,
    53028,
    53029,
    53030,
    53031,
    53032,
    53033,
    53034,
    53035,
    53036,
    35776,
    35777,
    35778,
    35779,
    35780,
    51707,
    51708,
    51709,
    51710,
    51711]},
  'tags': ['1-hop'],
  'RuBQ_version': '1.

### Смотрим для каких параграфов нет доступных вики-страниц

In [37]:
def get_no_reachable_paragraphs(paragraphs: List[dict], 
                                dst_dir: str='logs',
                                single_file_timeout: int = 30,
                                follow_redirects: bool = True
                                ):
    # We do not need files anymore
    if not os.path.isdir(dst_dir):
        os.mkdir(dst_dir)

    error_pages = []
    unavailable_paragraphs = []
    checked_pages_unavailable = set()
    checked_pages_available = set()

    pbar = tqdm(total=len(paragraphs))

    try:
        for item in paragraphs:
            wiki_page_id = item["ru_wiki_pageid"]
            wiki_url = f'https://ru.wikipedia.org/w/index.php?curid={wiki_page_id}'

            if wiki_page_id in checked_pages_unavailable:
                unavailable_paragraphs.append(item)
                pbar.update(1)
                continue
            elif wiki_page_id in checked_pages_available:
                pbar.update(1)  # page was successfully loaded - just continue
                continue

            # Only ping the page (HEAD is enough)
            try:
                resp = httpx.head(
                    wiki_url,
                    timeout=single_file_timeout,
                    follow_redirects=follow_redirects
                )
                resp.raise_for_status()
                checked_pages_available.add(wiki_page_id)

            except httpx.RequestError as req_exc:
                msg = f'request error for url={wiki_url}: {str(req_exc)}'
                print(msg, file=sys.stderr)
                error_pages.append({'url': wiki_url,
                                    'exc_type': 'request error',
                                    'msg': msg})
                unavailable_paragraphs.append(item)
                checked_pages_unavailable.add(wiki_page_id) # add pages to unavailables
            except httpx.HTTPStatusError as st_exc:
                msg = f'invalid status for url={wiki_url}: {str(st_exc)}'
                print(msg, file=sys.stderr)
                error_pages.append({'url': wiki_url,
                                    'exc_type': 'status error',
                                    'msg': msg})
                unavailable_paragraphs.append(item)
                checked_pages_unavailable.add(wiki_page_id) # add pages to unavailables
            pbar.update(1)

    except KeyboardInterrupt:
        print(f'Execution interrupted', file=sys.stderr)

    finally:
        # write exceptions info:
        with open(os.path.join(dst_dir, 'load_errors.json'), 'w', encoding='utf-8') as f:
            f.write(json.dumps(error_pages, ensure_ascii=False))

        with open(os.path.join(dst_dir, 'no_wiki_paragraphs.json'), 'w', encoding='utf-8') as f:
            f.write(json.dumps(unavailable_paragraphs, ensure_ascii=False))
    # return paragraphs that belong to unreachable pages
    return unavailable_paragraphs

In [22]:
no_reachable_paragraphs = get_no_reachable_paragraphs(paragraphs)
no_reachable_paragraphs

  4%|▎         | 2110/56952 [01:12<55:04, 16.60it/s]  invalid status for url=https://ru.wikipedia.org/w/index.php?curid=3682344: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=3682344'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
 13%|█▎        | 7543/56952 [05:45<24:25, 33.71it/s]  invalid status for url=https://ru.wikipedia.org/w/index.php?curid=5705399: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=5705399'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
 15%|█▍        | 8310/56952 [06:15<52:47, 15.36it/s]  invalid status for url=https://ru.wikipedia.org/w/index.php?curid=7538508: Client error '404 Not Found' for url 'https://ru.wikipedia.org/w/index.php?curid=7538508'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
 15%|█▍        | 8315/56952 [06:16<1:04:05, 12.65it/s]invalid status for url

[{'uid': 2113,
  'ru_wiki_pageid': 3682344,
  'text': 'Участник интеллектуальных игр Анатолий Вассерман считает необоснованными гипотезы о роли Хуго Шмайссера в создании автомата, и особо подчёркивает, что Михаил Тимофеевич Калашников — несомненно, талантливый изобретатель, на счету которого множество изобретений в сфере оружия и боевой техники, но при этом, заявляет, что в самом автомате, принятом на вооружение в 1947 году, не было ни одного изобретения Калашникова, и лишь впоследствии М. Т. Калашников получил патенты на усовершенствование некоторых узлов — патенты, которые не охватывали всей конструкции. Как утверждает Вассерман, непосредственно в конструкции 1947 года вообще ни одного калашниковского изобретения нет. И хотя главную роль в создании автомата сыграл всё-таки Калашников, это была роль не изобретателя, а именно конструктора. Калашников, согласно Вассерману, превосходно изучил множество других конструкций оружия, созданных к тому времени, когда он работал над созданием ав

In [23]:
len(no_reachable_paragraphs)

270

### Основные функции

In [38]:
def create_berta_embeddings(berta_model: AutoModel, 
                            berta_tokenizer: AutoTokenizer, 
                            inputs: list[str], 
                            batch_size=32, 
                            device: str='cuda',
                            prefix: str="search_document: "):
    def pool(hidden_state, mask, pooling_method="mean"):
        if pooling_method == "mean":
            s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
            d = mask.sum(axis=1, keepdim=True).float()
            return s / d
        elif pooling_method == "cls":
            return hidden_state[:, 0]

    # add task prefix if exists:
    if prefix:
        inputs = [prefix + input_str for input_str in inputs]

    batch_count = (len(inputs) + batch_size - 1) // batch_size
    result_embeddings = []
    pbar = tqdm(total=batch_count, desc='creating embeddings...')

    with torch.no_grad():
        for i in range(batch_count):
            batch = inputs[i*batch_size: (i + 1) * batch_size]
            tokenized_inputs = berta_tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors="pt").to(device)
            berta_model.to(device)
            outputs = berta_model(**tokenized_inputs)
            embeddings = pool(
                outputs.last_hidden_state, 
                tokenized_inputs["attention_mask"],
                pooling_method="mean"
            )

            embeddings = F.normalize(embeddings, p=2, dim=1).to('cpu')
            result_embeddings.append(embeddings)
            pbar.update(1)
    result_embeddings = torch.cat(result_embeddings, dim=0)
    print(f'embeddings count={len(result_embeddings)}', flush=True)
    
    return result_embeddings


def load_model(model_name:str="sergeyzh/BERTA"):
    print('loading model and tokenizer...', flush=True)
    try:
        berta_tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
        berta_model = AutoModel.from_pretrained(model_name, local_files_only=True)
    except Exception as e:
        print('no local files found, load from server...', flush=True)
        berta_tokenizer = AutoTokenizer.from_pretrained(model_name)
        berta_model = AutoModel.from_pretrained(model_name)

    return berta_model, berta_tokenizer


# split large paragraphs into chunks with smart-chunker
def chunk_large_paragraphs(paragraphs: dict, tokenizer: AutoTokenizer, max_tokens=512):
    chunker = SmartChunker(
        language='ru',
        reranker_name='BAAI/bge-reranker-v2-m3',
        newline_as_separator=False,
        device='cuda:0'
    )
    # split large paragraph into smaller
    def split_paragraph(paragraph):
        nonlocal chunker
        texts = chunker.split_into_chunks(paragraph['text'])

        return [{'uid':paragraph['uid'], 'text':text, 'ru_wiki_pageid': paragraph['ru_wiki_pageid']} for text in texts]

    result = []
    texts = [paragraph['text'] for paragraph in paragraphs]
    tokenized = tokenizer(texts, truncation=False)

    token_counts = [len(tokens) for tokens in tokenized['input_ids']]
    pbar=tqdm(total=len(paragraphs), desc='Chunking large paragraphs...')

    for i in range(len(paragraphs)):
        token_count = token_counts[i]
        paragraph = paragraphs[i]

        if token_count > max_tokens:
            new_paragraphs = split_paragraph(paragraph)
            for n_par in new_paragraphs:
                result.append(n_par)
        else:
            result.append(paragraph)

        pbar.update(1)

    return result


async def get_qdrant_paragraph(qdrant_client: AsyncQdrantClient, p_id: int, collection:str='paragraphs') -> dict:
    filtered = await qdrant_client.scroll(
        collection_name=collection,
        scroll_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="uid",
                        match=models.MatchValue(value=p_id),
                    )
                ]
        )
    )

    return filtered


async def create_paragraphs_database(qdrant_client: AsyncQdrantClient,
                                     paragraphs:dict,
                                     collection:str='paragraphs',
                                     batch_size:int=32,
                                     chunk_large:bool=False,
                                     cache_embeddings:bool=True,
                                     use_cached_embeddings:bool=False):
    if not qdrant_client:
        return
    # check collection already created
    if await qdrant_client.collection_exists(collection):
        return
    await qdrant_client.create_collection(collection_name=collection,
                                          vectors_config=models.VectorParams(size=EMBEDDINGS_DIM, distance=models.Distance.COSINE)
                                          )
    berta_model, berta_tokenizer = load_model()
    # chunk large paragraphs if required
    if chunk_large:
        paragraphs = chunk_large_paragraphs(paragraphs, berta_tokenizer)

    texts = [paragraph['text'] for paragraph in paragraphs]
    batch_size = 32

    if use_cached_embeddings:
        # load checkpoint embeddings:
        if not os.path.isfile('embeddings.pkl'):
            raise Exception('Trying to load cached embeddings - no cached embeddings file found')
        print("load cached embeddings...")
        with open('embeddings.pkl', 'rb') as f:
            embeddings = pickle.load(f)
    else: 
        # create from scratch:
        embeddings = create_berta_embeddings(berta_model, berta_tokenizer, texts, batch_size=batch_size)

    # save embeddings in case of cache
    if cache_embeddings and not use_cached_embeddings:
        with open('embeddings.pkl', 'wb') as f:
            pickle.dump(embeddings, f)

    id_counter = count(start=0)    # global chunk id in qdrant database

    # embeddings and corresponding paragraphs iterator:
    def batch_iterator(paragraphs, embeddings, batch_size=32):
        for i in range(0, len(paragraphs), batch_size):
            yield (paragraphs[i: i + batch_size], embeddings[i:i + batch_size])

    pabar=tqdm(total=(len(paragraphs) + batch_size - 1) // batch_size, desc='loading messages to Qdrant storage...')
    batch_idx=0

    # insert chunks to qdrant base:
    for batch in batch_iterator(paragraphs, embeddings, batch_size):
        paragraphs, embeddings = batch
        operation_info = await qdrant_client.upsert(collection_name=collection,
                                                    points=[models.PointStruct(id=next(id_counter),
                                                                                vector=embeddings[i],
                                                                                payload={"paragraph_id": paragraphs[i]['uid'], 
                                                                                         "text": paragraphs[i]['text']
                                                                                         }
                                                                                )
                                                            for i in range(len(embeddings))
                                                            ]
                                                    )
        if operation_info.status == models.UpdateStatus.ACKNOWLEDGED:
            print(f'WARNING: acknowledged request on batch - {batch_idx}', flush=True)
        batch_idx += 1
        pabar.update(1)

    berta_model.to('cpu')
    del berta_model


async def create_chunked_database(qdrant_client: AsyncQdrantClient,
                                  documents_dir: str='data/', 
                                  chunk_splitter:str='\n'*4, 
                                  collection:str='nsu_base'
                                  ):
    if not qdrant_client:
        return
    # check collection already created
    if await qdrant_client.collection_exists(collection):
        return
    await qdrant_client.create_collection(collection_name=collection,
                                          vectors_config=models.VectorParams(size=EMBEDDINGS_DIM, distance=models.Distance.COSINE)
                                          )
    berta_model, berta_tokenizer = load_model()
    id_counter = count(start=0)    # global chunk id in qdrant database

    for file in tqdm(filter(lambda f: is_text(f), os.listdir(documents_dir)), desc="inserting chunks..."):
        f_path = os.path.join(documents_dir, file)
        with open(f_path, encoding='utf-8') as f:
            data = f.read()

        # create embeddings of document's chunks:
        chunks = data.split(chunk_splitter)
        embeddings = create_berta_embeddings(berta_model, berta_tokenizer, chunks)
        local_id_counter = count(start=0)  # chunk id inside document

        # insert embeddings to qdrant base:
        operation_info = await qdrant_client.upsert(collection_name=collection,
                                                    points=[models.PointStruct(id=next(id_counter),
                                                                               vector=embeddings[i],
                                                                               payload={"doc_name": file, "chunk_id": next(local_id_counter), "text": chunks[i]}
                                                                               )
                                                            for i in range(len(embeddings))
                                                            ]
                                                    )
        if operation_info.status == models.UpdateStatus.ACKNOWLEDGED:
            print(f'WARNING: acknowledged request for doc - {file}', flush=True)
    berta_model.to('cpu')
    del berta_model


### Смотрим распределение числа токенов в параграфах

In [7]:
import plotly.graph_objects as go
from typing import List

berta_tokenizer = AutoTokenizer.from_pretrained("sergeyzh/BERTA", local_files_only=True)

def plot_tokens_dist(paragraphs, tokenizer):
    texts = [paragraph['text'] for paragraph in paragraphs]
# Example: suppose you have token counts for each text
    def get_tokens_distribution(tokenizer: AutoTokenizer, texts: List[str]):
        tokenized = tokenizer(texts, truncation=False)

        return [len(tokens) for tokens in tokenized['input_ids']]

    tokens_count = get_tokens_distribution(tokenizer, texts)

    fig = go.Figure(
        data=[go.Histogram(x=tokens_count, nbinsx=50, marker=dict(line=dict(width=1, color="black")))]
    )

    # threshold:
    fig.add_shape(
        type="line",
        x0=512,
        x1=512,
        y0=0,
        y1=max(tokens_count),  # or you can use fig.data[0].y.max() after rendering
        line=dict(color="red", width=2, dash="dash")
    )
    fig.update_layout(
        title="Token Count Distribution",
        xaxis_title="Number of Tokens",
        yaxis_title="Frequency",
        bargap=0.1,
    )    

    fig.show()

### Распределение числа токенов для исходных параграфов

In [8]:
plot_tokens_dist(paragraphs, berta_tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


### После чанкинга

In [9]:
load_chunks=True
if load_chunks:
    with open('doc_base/chunked_paragraphs.json', 'r', encoding='utf-8') as f:
        chunked_large_paragraphs = json.loads(f.read())
else:
    chunked_large_paragraphs = chunk_large_paragraphs(paragraphs, berta_tokenizer)

In [10]:
plot_tokens_dist(chunked_large_paragraphs, berta_tokenizer)

In [9]:
with open('doc_base/chunked_paragraphs.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(chunked_large_paragraphs))

In [11]:
chunked_large_paragraphs[0]

{'uid': 0,
 'ru_wiki_pageid': 58311,
 'text': 'ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).'}

### Посмотрим как отработало разбиение на чанки

In [12]:
def get_tokens_count(paragraphs: dict, tokenizer: AutoTokenizer):
    texts = [paragraph['text'] for paragraph in paragraphs]
# Example: suppose you have token counts for each text
    def get_tokens_distribution(tokenizer: AutoTokenizer, texts: List[str]):
        tokenized = tokenizer(texts, truncation=False)

        return [len(tokens) for tokens in tokenized['input_ids']]

    tokens_count = get_tokens_distribution(tokenizer, texts)

    return tokens_count

In [13]:
tokens_count = get_tokens_count(paragraphs, berta_tokenizer)
large_paragraphs = [paragraphs[i] for i in range(len(paragraphs)) if tokens_count[i] > 512]

st_pos=10
end_pos=10

for large in large_paragraphs[st_pos: end_pos + 1]:
    print('Original:\n')
    print(large['text'])
    print('#' * 40)
    print('Chunked:\n')

    splitted = [chunked for chunked in chunked_large_paragraphs if chunked['uid'] == large['uid']]

    for chunk in splitted:
        print(chunk['text'])
        print('\n' + '-'*40)

Original:

Валуйский приводит сведения из статьи Ш. Чухо в сочинской газете «Красное знамя» от 27 марта 1960 г. о том, что одна из форм названия р. Сочи — Сшычье (в исторических источниках неизвестная). По свидетельству Ш.Чухо, в старошапсугском диалекте сши означает «брат», а чье — «без», то есть «река без притока». Однако такой перевод относится к образцу «народных этимологий». Если переводить с шапсугского диалекта, то надо исходить и из шапсугских же вариантов названия: Сша-ше, Шаше, Сфеши. К тому же река Сочи уже в устье имеет правый приток с широкой долиной, ручей Хлудовского (Шлабистага), не говоря о других (Ац, Агва, Ушха, Ажек и пр.). М. В. Валуйский сообщает также, что у черноморских шапсугов было распространено неправильное мнение (основанное на сходстве звучания) о том, что название Сочи связано с адыгейским наименованием конных скачек, так называемых Шъаче (на самом деле по-адыг. скачки — шыгъачьэ), которые прежде осуществлялись на обширной равнине левого приустья реки Соч

### Создаём Qdrant-client и базу

In [39]:
base_name = 'paragraphs'
query_prefix = "search_query: "
qdrant_client = AsyncQdrantClient(url=f"http://localhost:{REST_API_PORT}")

In [40]:
qdrant_client

In [41]:
await create_paragraphs_database(qdrant_client, paragraphs, batch_size=64, use_cached_embeddings=True)
vector_store = QdrantVectorStore(aclient=qdrant_client, collection_name=base_name)

### Задаём свою эмбеддер-модель

In [42]:
# add prefix for query embeddings calculation:
embed_model = HuggingFaceEmbedding(model_name="sergeyzh/BERTA", device='cuda', query_instruction=query_prefix)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, 
                                           embed_model=embed_model
                                          )

Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


### Доступ к llm по openAI API

In [ ]:
from llama_index.llms.openai_like import OpenAILike


llm = OpenAILike(
    model=f"gpt://{YANDEX_CLOUD_FOLDER}/{model_name}",
    api_base="https://llm.api.cloud.yandex.net/v1",
    api_key=YANDEX_CLOUD_API_KEY,
    is_chat_model=True  # set as needed for your model
)
#query_engine = index.as_query_engine(llm=llm)

In [62]:
# Вариант с Open Router
from llama_index.llms.openrouter import OpenRouter

open_router_token_path = 'token'

with open(open_router_token_path, 'r', encoding='utf-8') as f:
    open_router_token = f.read()

MAX_TOKENS = 1024
CONTEXT = 4096
ENGINE_MODEL = 'qwen/qwen3-14b'

open_router_llm = OpenRouter(
    api_key = open_router_token,
    max_tokens = MAX_TOKENS,
    context_window = CONTEXT,
    model = ENGINE_MODEL,
    additional_kwargs = {
        "extra_body": {
            "provider": {
                "only": ["DeepInfra"],
                "quantizations": ["fp8"],
                "allow_fallbacks": False,
                "require_parameters": True
                }
        }
    }
)

In [63]:
from IPython.display import Markdown, display


# define prompt viewing function
def display_prompts_dict(prompts_dict):
    for k, p in prompts_dict.items():
        text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
        display(Markdown(text_md))
        print(p.get_template())
        display(Markdown("<br><br>"))

In [64]:
query_engine = index.as_query_engine(llm=open_router_llm)

In [65]:
prompts_dict = query_engine.get_prompts()
display_prompts_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

### Кастомизируем промпт

In [ ]:
from llama_index.core.prompts import PromptTemplate

ru_qa_prompt_str = (
    "Ты вопрос-ответный ассистент на русском языке. Ты не используешь в своих ответах никакого Markdown синтаксиса (никаких звёздочек, решёток, списков и т.д.). По возможности отвечай кратко.\n"
    "Ниже приведена контекстная информация:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Ответь на запрос, опираясь только на предоставленный контекст и не используя внешние знания.\n"
    "Запрос: {query_str}\n"
)
ru_refine_prompt_str = (
    "Имеется исходный запрос: {query_str}\n"
    "Имеется существующий ответ: {existing_answer}\n"
    "Ниже приведён дополнительный контекст, который может помочь уточнить ответ.\n"
    "---------------------\n"
    "{context_msg}\n"
    "---------------------\n"
    "Используя этот новый контекст, уточни первоначальный ответ, чтобы лучше раскрыть запрос.\n"
    "Если дополнительный контекст не помогает, верни исходный существующий ответ.\n"
)
ru_qa_prompt = PromptTemplate(ru_qa_prompt_str)
ru_refine_prompt = PromptTemplate(ru_refine_prompt_str)

In [72]:
query_engine.update_prompts({"response_synthesizer:text_qa_template":ru_qa_prompt, "response_synthesizer:refine_template":ru_refine_prompt})

In [73]:
prompts_dict = query_engine.get_prompts()
display_prompts_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Ты вопрос-ответный ассистент на русском языке. Ты не используешь в своих ответах никакого Markdown синтаксиса (никаких звёздочек, решёток, списков и т.д.).
Ниже приведена контекстная информация:
---------------------
{context_str}
---------------------
Ответь на запрос, опираясь только на предоставленный контекст и не используя внешние знания.
Запрос: {query_str}



<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

Имеется исходный запрос: {query_str}
Имеется существующий ответ: {existing_answer}
Ниже приведён дополнительный контекст, который может помочь уточнить ответ.
---------------------
{context_msg}
---------------------
Используя этот новый контекст, уточни первоначальный ответ, чтобы лучше раскрыть запрос.
Если дополнительный контекст не помогает, верни исходный существующий ответ.



<br><br>

### Тестируем наш RAG

In [74]:
queries_count=10
queries = [query['question_text'] for query in dev_queries[:queries_count]]

In [75]:
for query in queries:
    print(f'Question: {query}')
    resp = await query_engine.aquery(query)
    print(f'Response: {str(resp)}')
    print('-'*40)

Question: Какой стране принадлежит знаменитый остров Пасхи?
Response: Остров Пасхи (Рапа-Нуи) принадлежит Чили. В 1888 году остров был аннексирован этой страной, а в 2018 году власти Чили ускорили процесс переименования острова в Рапа-Нуи.
----------------------------------------
Question: С какой музыкальной группой неразрывно связано имя Мика Джаггера?
Response: Имя Мика Джаггера неразрывно связано с рок-группой The Rolling Stones.
----------------------------------------
Question: Где находится Летний сад?
Response: Летний сад находится в Центральном районе Санкт-Петербурга, согласно первому источнику. Однако второй источник указывает, что он расположен в историческом центре Кронштадта, на Петровской улице. Эти данные противоречат друг другу, но в рамках предоставленного контекста оба утверждения приведены.
----------------------------------------
Question: Какой город является столицей Туркмении?
Response: Столицей Туркмении является город Ашхабад.
---------------------------------

In [ ]:
# prompts = query_engine.get_prompts()
# prompts

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x776340b86d40>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

### Логирование

In [13]:
def log_resp(resp):
    selected_chunks = resp.source_nodes
    print(f"chunks count={len(selected_chunks)}")
    
    for chunk in selected_chunks:
        metadata = chunk.metadata
        doc_name = metadata.get("doc_name")
        chunk_id = metadata.get("chunk_id")
        chunk_text = chunk.text
        
        # Log or print the chunk information
        print(f"Document Name: {doc_name}, Chunk ID: {chunk_id}, Chunk Text: \n{chunk_text}") 
        print('-'*50)

In [14]:
log_resp(resp)

chunks count=2
Document Name: None, Chunk ID: None, Chunk Text: 
Сэ́мюэл Фи́нли Бриз Мо́рзе (англ. Samuel Finley Breese Morse [mɔːrs]; 27 апреля 1791, Чарльзтаун в штате Массачусетс — 2 апреля 1872, Нью-Йорк) — американский изобретатель и художник. Наиболее известные изобретения — электромагнитный пишущий телеграф («аппарат Морзе», 1836) и код (азбука) Морзе.
--------------------------------------------------
Document Name: None, Chunk ID: None, Chunk Text: 
Газеты, железные дороги и банки быстро нашли применение его телеграфу. Телеграфные линии моментально оплели весь мир, состояние и слава Морзе умножились. В 1858 году от десяти европейских государств Морзе получил за своё изобретение 400 000 франков. Морзе купил имение в Покипси, близ Нью-Йорка, и провёл там остаток жизни с большим семейством среди детей и внуков. В старости Морзе стал филантропом. Он опекал школы, университеты, церкви, библейские общества, миссионеров и бедных художников.
-------------------------------------------

### Бенчмаркинг с RAGAS

In [24]:
import numpy as np
from ragas.metrics import DiscreteMetric
from ragas import experiment
import asyncio
from ragas.metrics import FactualCorrectness
import os
import sys

In [94]:
def create_test_bench(bench_size, queries: list[dict], random_state = 42, skip_queries:set={}):
    # filter bad queries: no answer paragraphs or in skip_queries
    queries = [q for q in queries if len(q['paragraphs_uids']['with_answer']) > 0 and q['uid'] not in skip_queries]
    np.random.seed(seed=random_state)
    idxs = np.arange(0, len(queries))
    np.random.shuffle(idxs) # shuffle inplace

    bench_queries = np.array(queries)[idxs][:bench_size]
    result = []

    for i, query in enumerate(bench_queries):
        item = {'uid' : query['uid'],
                'text': query['question_text'],
                'answers': [item['label'] for item in query['answers']],
                'answer_text': query['answer_text'],
                'answer_paragraphs': query['paragraphs_uids']['with_answer']
                }
        result.append(item)

    return result

In [95]:
import json


# get paragraphs to remove:
with open('logs/no_wiki_paragraphs.json', 'r', encoding='utf-8') as f:
    no_wiki_paragraphs = json.loads(f.read())
no_wiki_pids = {par['uid'] for par in no_wiki_paragraphs}


def get_no_wiki_queries(no_wiki_pids, queries):
    answer = set()

    for query in queries:
        answer_paragraphs = set(query['paragraphs_uids']['with_answer'])
        if answer_paragraphs.issubset(no_wiki_pids):
            answer.add(query['uid'])

    return answer

In [96]:
no_wiki_dev_queries = get_no_wiki_queries(no_wiki_pids, dev_queries)
no_wiki_test_queries = get_no_wiki_queries(no_wiki_pids, test_queries)
print(f'No wiki queries count={len(no_wiki_dev_queries)} from total={len(dev_queries)} queries')
print(f'No wiki queries count={len(no_wiki_test_queries)} from total={len(test_queries)} queries')

No wiki queries count=137 from total=580 queries
No wiki queries count=639 from total=2330 queries


In [97]:
test_bench_size = 20
test_bench = create_test_bench(test_bench_size, dev_queries, skip_queries=no_wiki_dev_queries)
test_bench

[{'uid': 6669,
  'text': 'В каком году изобретен велосипед?',
  'answers': ['1885'],
  'answer_text': 'В 1817 году',
  'answer_paragraphs': [20831]},
 {'uid': 745,
  'text': 'Кто является автором пьесы о легендарном античном мизантропе «Жизнь Тимона Афинского»?',
  'answers': ['Уильям Шекспир'],
  'answer_text': 'Уильям Шекспир',
  'answer_paragraphs': [55161, 55162]},
 {'uid': 343,
  'text': 'Кто из пиратов плавал на корабле «Золотая лань»?',
  'answers': ['Дрейк, Фрэнсис'],
  'answer_text': 'Фрэнсис Дрейк',
  'answer_paragraphs': [54013, 54014]},
 {'uid': 6865,
  'text': 'В каком году образована Калининградская область?',
  'answers': ['1946'],
  'answer_text': '1946',
  'answer_paragraphs': [25444]},
 {'uid': 344,
  'text': 'Как звали дочь Иродиады, танец которой вынудил Ирода отдать приказ обезглавить Иоанна Крестителя?',
  'answers': ['Саломея'],
  'answer_text': 'Саломея',
  'answer_paragraphs': [54038]},
 {'uid': 6421,
  'text': 'Какая киностудия снимает фильмы Марвел?',
  'answ

#### Тест Qwen3-14B

In [28]:
from openai import OpenAI

open_router_token_path = 'token'

with open(open_router_token_path, 'r', encoding='utf-8') as f:
    open_router_token = f.read()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=open_router_token,
)

completion = client.chat.completions.create(
    #model="qwen/qwen3-next-80b-a3b-instruct",
    model="qwen/qwen3-14b",
    messages=[
        {"role": "user", "content": "Привет! А что ты умеешь?"}
    ],
    max_completion_tokens=1024,
    extra_body={
        "provider": {
            #"order": ["Hyperbolic", "GMICloud", "DeepInfra"],
#            "only":["Hyperbolic"],
#            "quantizations": ["bf16"],
            "only":["DeepInfra"],
            "quantizations": ["fp8"],
            "allow_fallbacks": False,
            "require_parameters": True
        }
    }
)
print(completion.choices[0].message.content)

Привет! 😊 Я – Qwen, крупнейшая языковая модель Alibaba Cloud. Я могу помочь вам в следующих аспектах:

1. **Всеобъемлющие ответы на вопросы** – от науки до культуры, от истории до современных технологий.
2. **Написание текстов**: статьи, рассказы, письма, email, технические документы и многое другое.
3. **Изучение языков**: я могу помочь вам улучшить английский, китайский, испанский и другие языки.
4. **Программирование**: помощь в написании кода, объяснении алгоритмов, отладке программ.
5. **Математика**: решение задач, объяснение теорем, помощь в подготовке к экзаменам.
6. **Творчество**: генерация идей, создание концептов, помощь в художественном процессе.
7. **Наставничество и планирование**: помочь с постановкой целей, составлением планов, развитием навыков.
8. **Рекомендации**: книги, фильмы, путешествия, продукты и многое другое.
9. **Поддержка нескольких языков** – включая китайский, английский, русский и другие.

Если у вас есть какие-либо вопросы или задачи, не стесняйтесь за

#### Тест Gemma3-27B-it

In [31]:
from openai import OpenAI

open_router_token_path = 'token'

with open(open_router_token_path, 'r', encoding='utf-8') as f:
    open_router_token = f.read()

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=open_router_token,
)

completion = client.chat.completions.create(
    #model="qwen/qwen3-next-80b-a3b-instruct",
    model="google/gemma-3-27b-it",
    messages=[
        {"role": "user", "content": "Привет! А что ты умеешь?"}
    ],
    max_completion_tokens=1024,
    extra_body={
        "provider": {
            #"order": ["Hyperbolic", "GMICloud", "DeepInfra"],
#            "only":["Hyperbolic"],
#            "quantizations": ["bf16"],
            #"only":["NovitaAI", "Nebius AI Studio"],
            "quantizations": ["bf16"],
            "allow_fallbacks": False,
            "require_parameters": True
        }
    }
)
print(completion.choices[0].message.content)



Привет! Я — большая языковая модель Gemma, разработанная командой Google DeepMind. Я специализируюсь на обработке текста и могу выполнять разные задачи, например:

*   **Генерация текста:** Я могу писать тексты разных стилей и форматов: рассказы, стихи, сценарии, письма, статьи и т.д.
*   **Перевод:** Я могу переводить тексты с одного языка на другой.
*   **Ответы на вопросы:** Я могу отвечать на вопросы, используя свои знания и информацию, полученную из текста.
*   **Обобщение:** Я могу сокращать длинные тексты, выделяя основную информацию.
*   **Классификация:** Я могу определять тему или категорию текста.
*   **Разговор:** Я могу поддерживать беседу на разные темы.

Я пока ещё в стадии разработки, но постоянно учусь и совершенствуюсь.

Я являюсь моделью с открытым весом, что означает, что я широко доступна для использования. Мои создатели - команда Gemma.



### Определяем LLM для метрик

In [ ]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.run_config import RunConfig


MAX_EVAL_TOKENS = 2048
RESPONSE_TIMEOUT = 120
open_router_token_path = 'token'

with open(open_router_token_path, 'r', encoding='utf-8') as f:
    open_router_token = f.read()

# Initialize LangChain LLM using OpenRouter endpoint
evaluator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model="qwen/qwen3-next-80b-a3b-instruct",
        openai_api_key=open_router_token,
        openai_api_base="https://openrouter.ai/api/v1",  # important!
        max_completion_tokens=MAX_EVAL_TOKENS,
        max_retries=2,
        timeout=RESPONSE_TIMEOUT,
        # default_headers={
        #     "HTTP-Referer": "https://your-app-name.com",  # optional but recommended
        #     "X-Title": "Ragas Evaluation",                 # optional
        # },
        extra_body={
            "provider": {
                "only": ["DeepInfra"],
                "quantizations": ["bf16"],
                "allow_fallbacks": False,
                "require_parameters": True,
            }
        }
    )
)

eval_run_config = RunConfig(max_workers=1, max_retries=1)



/tmp/ipykernel_61261/1258215674.py:14: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = LangchainLLMWrapper(


### Определяем LLM для ответов

In [ ]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper


MAX_GEN_TOKENS = 1024
RESPONSE_TIMEOUT = 120
open_router_token_path = 'token'

with open(open_router_token_path, 'r', encoding='utf-8') as f:
    open_router_token = f.read()

# Initialize LangChain LLM using OpenRouter endpoint
evaluator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model="qwen/qwen3-14b",
        openai_api_key=open_router_token,
        openai_api_base="https://openrouter.ai/api/v1",  # important!
        max_completion_tokens=MAX_EVAL_TOKENS,
        max_retries=2,
        timeout=RESPONSE_TIMEOUT,
        # default_headers={
        #     "HTTP-Referer": "https://your-app-name.com",  # optional but recommended
        #     "X-Title": "Ragas Evaluation",                 # optional
        # },
        extra_body={
            "provider": {
                "only": ["DeepInfra"],
                "quantizations": ["fp8"],   # only fp8 available
                "allow_fallbacks": False,
                "require_parameters": True,
            }
        }
    )
)



In [ ]:
from ragas import SingleTurnSample
from ragas.metrics import LLMContextPrecisionWithoutReference
from ragas.metrics._factual_correctness import FactualCorrectness
from ragas.metrics import Faithfulness
from ragas.metrics import LLMContextRecall
import json
from tqdm import tqdm


async def evaluated_experiment(query, query_engine, evaluator_llm, experiment_name:str='demo'):
    context = None
    query_text = None
    try:
        query_text = query['text']
        response = await query_engine.aquery(query_text)

        response_text = str(response)
        selected_chunks = response.source_nodes
        context = [chunk.text for chunk in selected_chunks]
        expected_answer = query['answer_text']
        # 
        factual_correctness = FactualCorrectness(llm=evaluator_llm)
        sample = SingleTurnSample(
            response=response_text,
            reference=expected_answer
        )
        factual_correctness_score = await factual_correctness.single_turn_ascore(sample)
        #
        context_precision = LLMContextPrecisionWithoutReference(llm=evaluator_llm)
        sample = SingleTurnSample(
            user_input=query_text,
            response=response_text,
            retrieved_contexts=context,
        )
        context_precision_score = await context_precision.single_turn_ascore(sample)
        # 
        faithfulness = Faithfulness(llm=evaluator_llm)
        faithfulness_score = await faithfulness.single_turn_ascore(sample)
        #
        sample = SingleTurnSample(
            user_input=query_text,
            response=response_text,
            reference=expected_answer,
            retrieved_contexts=context,
        )

        context_recall = LLMContextRecall(llm=evaluator_llm)
        context_recall_score = await context_recall.single_turn_ascore(sample)

    except Exception as e:
        exception_str = f'exception at query {query['uid']}: {query['text']} - {str(e)}'
        print(exception_str, file=sys.stderr)

        return {'exception': exception_str}

    return {
        "query": query_text,
        "response": response_text,
        "context": context,
        "expected_answer": expected_answer,
        "expected_paragraph": "",
        "factual_correctness": factual_correctness_score,
        "context_recall": context_recall_score,
        "context_precision": context_precision_score,
        "faithfulness": faithfulness_score
    }


async def run_benchmark(bench_data, 
                        query_engine, 
                        evaluator_llm, 
                        save_results: bool = True, 
                        dst_dir: str = '.', 
                        bench_name: str = 'demo',
                        num_coros: int = 8
                        ):
    results = {'scores': None, 'experiments':[]}
    experiments = []

    successfully_benches = 0
    factual_correctness_score = 0
    context_recall_score = 0
    context_precision_score = 0
    faithfulness_score = 0
    pbar = tqdm(total=len(bench_data))

    for bench_item in bench_data:
        result = await evaluated_experiment(bench_item, query_engine, evaluator_llm)

        if 'exception' not in result:
            successfully_benches += 1
            factual_correctness_score += result['factual_correctness']
            context_recall_score += result['context_recall']
            context_precision_score += result['context_precision']
            faithfulness_score += result['faithfulness']

        experiments.append({'uid': bench_item['uid'], 'result': result})
        pbar.update(1)

    results['experiments'] = experiments
    results['scores'] = {'factual_correctness': float(factual_correctness_score / successfully_benches),
                         'context_recall': float(context_recall_score / successfully_benches), 
                         'context_precision': float(context_precision_score / successfully_benches), 
                         'faithfulness': float(faithfulness_score / successfully_benches)
                         }
    if save_results:
        with open(os.path.join(dst_dir, f'{bench_name}.json'), 'w', encoding='utf-8') as f:
            f.write(json.dumps(results))

    return results


In [23]:
results = await run_benchmark(test_bench, query_engine, evaluator_llm)
results

100%|██████████| 10/10 [07:50<00:00, 47.09s/it]


{'scores': {'factual_correctness': np.float64(0.261),
  'context_recall': 0.5,
  'context_precision': 0.549999999965,
  'faithfulness': 0.9},
 'experiments': [{'uid': 8170,
   'result': {'query': 'Как зовут отца Витаса?',
    'response': 'Миколас Эдмундас Орбакас.',
    'context': ['Отец — Леопольд Витольдович Ростропович (1892—1942), виолончелист, педагог и дирижёр.',
     'Отец — артист цирка Миколас Эдмундас Орбакас.'],
    'expected_answer': 'Владас Аркадьевич Грачёв',
    'factual_correctness': np.float64(0.0),
    'context_recall': 0.0,
    'context_precision': 0.49999999995,
    'faithfulness': 0.0}},
  {'uid': 5184,
   'result': {'query': 'Какой показатель для воды при 20°С равен 81?',
    'response': 'I am sorry, but this question cannot be answered from the given text. The provided text discusses the characteristics of 20 ruble coins (material, weight, diameter) and the percentage of plastic in something, but does not contain information about water or any properties measured

In [ ]:
p_id = 5184
get_qdrant_paragraph(qdrant_client, p_id)